<a href="https://colab.research.google.com/github/Franklingo13/ejemplosCopernicus/blob/main/ejemplosJupyter/riesgoErosion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enfoque basado en el NDVI para estudiar áreas de deslizamientos de tierra
Las investigaciones indican que el NDVI puede desempeñar un papel crucial en la identificación de zonas de deslizamientos. Si bien es posible un flujo de trabajo avanzado, este cuaderno opta por un enfoque simple que utiliza la diferencia del NDVI y aplica un umbral al resultado para detectar zonas de deslizamientos. Nuestro análisis se basa en la colección Sentinel-2 Nivel 2A, obtenida del ecosistema de datos de Copernicus mediante openEO.

In [ ]:
%pip install openeo

In [ ]:
import openeo
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.patches as mpatches
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

In [ ]:
# Area de interés
spatial_extent = {
    "west": 13.882567409197492,
    "south": 40.7150627793427,
    "east": 13.928593792166282,
    "north": 40.747050251559216,
}

# Cargar la colección de datos Sentinel-2
s2pre = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=["2022-08-25", "2022-11-25"],
    spatial_extent=spatial_extent,
    bands=["B04", "B08"],
)
# Calcular el NDVI
prendvi = s2pre.ndvi().mean_time()

# Para el cubo de datos posterior, se carga la colección de datos Sentinel-2 con la extensión 
# temporal comenzando desde el final de la extensión temporal previa al evento.
s2post = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=["2022-11-26", "2022-12-25"],
    spatial_extent=spatial_extent,
    bands=["B04", "B08"],
)

# Calcular el NDVI para el cubo de datos posterior
postndvi = s2post.ndvi().mean_time()
# Calcular la diferencia del NDVI
diff = postndvi - prendvi
# Ejecuta el proceso
diff.download("NDVIDiff.tiff")


In [ ]:
# Graficar el resultado
# Cargar la imagen resultante
img = rasterio.open("NDVIDiff.tiff")

# Se aplica un umbral para resaltar las áreas de deslizamiento
value = img.read(1)
cmap = matplotlib.colors.ListedColormap(["black", "red"])
fig, ax = plt.subplots(figsize=(8, 6), dpi=80)
im = show(
    ((value < -0.48) & (value > -1)),
    vmin=0,
    vmax=1,
    cmap=cmap,
    transform=img.transform,
    ax=ax,
)
values = ["Ausencia", "Presencia"]
colors = ["black", "red"]
ax.set_title("Detected Landslide Area")
ax.set_xlabel("X Coordinates")
ax.set_ylabel("Y Coordinates")
patches = [
    mpatches.Patch(color=colors[i], label="Deslizamiento: {l}".format(l=values[i]))
    for i in range(len(values))
]
fig.legend(handles=patches, bbox_to_anchor=(0.83, 1.03), loc=1)